# Personal Art Creator App

This notebook trains a style model on your images and then launches a simple user interface to transform new photos.

## 1. Setup Environment

In [ ]:
# Install necessary libraries
!pip install -q diffusers transformers accelerate peft gradio
!pip install -q git+https://github.com/huggingface/diffusers.git

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

## 2. (First Time Only) Train The Model

**You only need to run this section ONCE.** Once the model is trained and saved, you can skip this entire section in the future.

### 2.1. Define Paths and Parameters

In [ ]:
import os

# --- IMPORTANT: VERIFY THIS PATH ---
# This should be the direct path to the folder in your Google Drive containing your training images.
SOURCE_IMAGE_FOLDER = "/content/drive/MyDrive/judit_art_training" # <-- PLEASE VERIFY THIS!

# --- Leave these as they are ---
# We copy images to a temporary local folder for faster processing
LOCAL_TRAINING_DIR = "/content/training_images_processed"
# This is where your final, permanent model will be saved
PERMANENT_MODEL_DIR_IN_DRIVE = "/content/drive/MyDrive/MyArtStyleModel"

# Training Parameters
BASE_MODEL_ID = "runwayml/stable-diffusion-v1-5"
INSTANCE_PROMPT = "a drawing in sks style" # The trigger for your style

### 2.2. Prepare Images for Training

In [ ]:
import shutil
from PIL import Image

# Create/clean the local directory
if os.path.exists(LOCAL_TRAINING_DIR):
    shutil.rmtree(LOCAL_TRAINING_DIR)
os.makedirs(LOCAL_TRAINING_DIR)

print("Copying and processing images...")
image_files = [f for f in os.listdir(SOURCE_IMAGE_FOLDER) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for filename in image_files:
    shutil.copy(os.path.join(SOURCE_IMAGE_FOLDER, filename), LOCAL_TRAINING_DIR)
    
    caption_filename = os.path.splitext(filename)[0] + ".txt"
    with open(os.path.join(LOCAL_TRAINING_DIR, caption_filename), "w") as f:
        f.write(INSTANCE_PROMPT)
        
    image_path = os.path.join(LOCAL_TRAINING_DIR, filename)
    try:
        with Image.open(image_path) as img:
            img = img.convert("RGB").resize((512, 512), Image.LANCZOS)
            img.save(image_path)
    except Exception as e:
        print(f"Could not process {filename}: {e}")

print(f"Processed {len(image_files)} images in {LOCAL_TRAINING_DIR}")

### 2.3. Train!

In [ ]:
# Download the training script
!wget -q -O train_dreambooth_lora.py https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora.py

# Launch Training
!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$BASE_MODEL_ID \
  --instance_data_dir=$LOCAL_TRAINING_DIR \
  --output_dir=$PERMANENT_MODEL_DIR_IN_DRIVE \
  --instance_prompt="$INSTANCE_PROMPT" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=500 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1500 \
  --seed=42

print(f'\n\nTraining finished! Your permanent model is saved in: {PERMANENT_MODEL_DIR_IN_DRIVE}')

---
## 3. (Future Use) Launch The App

**Once your model is trained, you can start here every time!** Just run the cells below.

In [ ]:
# Define paths (make sure these match section 2.1)
BASE_MODEL_ID = "runwayml/stable-diffusion-v1-5"
PERMANENT_MODEL_DIR_IN_DRIVE = "/content/drive/MyDrive/MyArtStyleModel"

In [ ]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import gradio as gr

# Load the main model and apply your custom style on top
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(BASE_MODEL_ID, torch_dtype=torch.float16).to("cuda")
pipe.load_lora_weights(PERMANENT_MODEL_DIR_IN_DRIVE)

print("Model loaded. Ready to create art!")

In [ ]:
def transform_image(input_image, style_strength):
    """This function takes an image and a strength value, and returns the transformed image."""
    if input_image is None:
        return None
    
    processed_image = input_image.resize((512, 512))
    
    prompt = "a drawing in sks style, high quality illustration, clear facial features"
    
    generator = torch.Generator("cuda").manual_seed(42)
    
    with torch.no_grad():
        result_image = pipe(
            prompt=prompt,
            image=processed_image,
            strength=style_strength,
            guidance_scale=7.5,
            generator=generator
        ).images[0]
        
    return result_image

# Create the Gradio interface
with gr.Blocks() as app:
    gr.Markdown("## Personal Art Style Transformer")
    gr.Markdown("Upload a photo to transform it into your unique art style.")
    with gr.Row():
        with gr.Column():
            source_image = gr.Image(type="pil", label="Source Photo")
            strength_slider = gr.Slider(minimum=0.5, maximum=1.0, step=0.05, value=0.75, label="Style Strength (higher value = more stylized)")
            submit_button = gr.Button("Transform")
        with gr.Column():
            output_image = gr.Image(label="Generated Illustration")
    
    submit_button.click(transform_image, inputs=[source_image, strength_slider], outputs=output_image)

# Launch the app!
app.launch(share=True, debug=True)